The following script calculates the membrane contact probabilities of each residue of a protein's cytosolic domain. This is for a membrane protein in a membrane system, with a cytosolic domain extending beyond the lower leaflet.

In [1]:
# Import the required modules
import MDAnalysis as mda
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# Load the structure file and trajectory and create the MDAnalysis Universe

PSF = 'protein.psf' # Assuming the structure file is in the current directory
XTCs = [
    'run1_centered.xtc',
    'run2_centered.xtc',
] # Loading two replicate runs, centered and autoimaged using CPPTRAJ


In [3]:
all_contact_counts = []
nframes = []
start = 1000 # Specify the starting frame for analysis, in case the first few frames need to be discarded.
step = 1

The following code assumes:<br>
* segment id of the protein in PROA,<br>
* the cytosolic domain is formed of residues 260 through 290<br>
* the waters and ions are stripped from the trajectory, and only the protein and the membrane are present<br>
* The lipids in the membrane all start with POP (POPC, POPS, POPI)

In [ ]:
for XTC in XTCs:
    u = mda.Universe(PSF, XTC)
    # Define a subselection to only the residues of interest, lipid atoms of the lower part of the membrane, and only heavy atoms (no H)
    u_sub = u.select_atoms('((segid PROA and resid 260 to 290) or (resname POP* and prop z < -12)) and not name H*')
    protein_icd = u.select_atoms('segid PROA and resid 260 to 290')
    nframes.append(u.trajectory.n_frames)

    # Precompute the selection for each residue once
    membrane_contact = {
        resid: u_sub.select_atoms(f'resname POP* and around 3.5 (segid PROB and resid {resid})', # Set the custom lipid selection
                                  updating=True) 
        for resid in protein_icd.residues.resids
    }
    
    # Accumulate the counts in an array
    residues = protein_icd.residues.resids
    contact_count_array = np.zeros(len(residues), dtype=int)
    
    # Iterate over each frame and compute contacts
    for ts in tqdm(u.trajectory[start::step]):
        # Create a temporary array for current frame
        frame_contacts = np.zeros(len(residues), dtype=int)
        
        # Update the frame_contacts array in one pass
        for idx, (resid, sel) in enumerate(membrane_contact.items()):
            if sel.n_atoms > 0:
                frame_contacts[idx] = 1
        
        # Accumulate frame contacts into the main contact count array
        contact_count_array += frame_contacts

    # Append the contact count array for this trajectory to the all contacts list
    all_contact_counts.append(contact_count_array)


In [ ]:
# Calculate the contact frequencies (contact count/number of frames)
contact_freqs = [
    all_contact_counts[0]/(nframes[0]-start),
    all_contact_counts[1]/(nframes[1]-start),
]

contact_freqs_array = np.array(contact_freqs)
contact_freqs_array

In [ ]:
# Save the contact frequencies into a CSV file

contact_freq_df = pd.DataFrame(
    contact_freqs_array,
    columns=residues, 
    index=[f'RUN_{i+1}' for i in range(len(XTCs))]
).transpose()

contact_freq_df.to_csv('membrane_contact_freq_per_residue.csv', header=True)
contact_freq_df